# Ingestando embeddings a Pinecone

In [5]:
#%%capture
#!pip install pinecone
#!pip install tiktoken

In [2]:
# import pinecone
from pinecone import Pinecone, ServerlessSpec
import os, getpass
import tiktoken

In [3]:
api_key = getpass.getpass("Ingresa tu API Key de Pinecone : ")

## Creando un index en Pinecone

In [5]:
from pinecone import Pinecone, ServerlessSpec
import os

# Configura la clave API de Pinecone en la variable de entorno
api_key = "pcsk_5Yvurj_JULv3S2PByfWme4XMWfwqvr1yBUbhQpj95HDutEmVtVqEUCZ1YWXRTpA2vPqPxd"
os.environ["PINECONE_API_KEY"] = api_key

# Nombre del índice que deseas crear
index_name = "knowledge-base-eliminatorias"

# Dimension del modelo, ajustado para Stable Diffusion
dimension = 1536

# Especificaciones del servidor donde se alojará el índice
spec = ServerlessSpec(
    cloud="aws",  # Ajusta según la nube que estés utilizando, "aws", "gcp", etc.
    region="us-east-1"  # Asegúrate de que la región esté disponible en tu plan
)

# Inicializa el cliente de Pinecone
pinecone_client = Pinecone()

# Crea un índice en Pinecone
pinecone_client.create_index(
    name=index_name, 
    dimension=dimension,
    spec=spec  # Incluye las especificaciones del servidor
)

print("Index " + index_name + " creado con éxito en Pinecone")

Index knowledge-base-eliminatorias creado con éxito en Pinecone


In [6]:
# *****************************************
# ANTIGUO
# *****************************************

#index_name = "knowledge-base-eliminatorias"
#dimension = 1536

# Conectarse con la única región permitida
#pc = Pinecone(api_key=api_key)

#if index_name not in [idx.name for idx in pc.list_indexes()]:
#    pc.create_index(
#        name=index_name,
#        dimension=dimension,
#        metric="cosine",
#        spec=ServerlessSpec(
#            cloud="aws",          # ✅ Solo AWS
#            region="us-east-1"    # ✅ Solo us-east-1
#        )
#    )
#    print(f"✅ Índice '{index_name}' creado correctamente.")
#else:
#    print(f"ℹ️ El índice '{index_name}' ya existe.")


## Generando fragmentos de sitios web

![Imagen](https://imgmedia.larepublica.pe/640x371/larepublica/original/2023/10/17/652f24917388f967a923a805.webp)

In [7]:
#!pip install -U langchain-community

In [8]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader(
    [
    "https://www.marca.com/co/2023/10/17/652e070f22601d73648b4585.html", 
    "https://hiraoka.com.pe/blog/post/eliminatorias-sudamericanas-mundial-2026-calendario-partidos-y-fechas"
    ]
)
data = loader.load()

#Genera varios fragmentos de 400 tokens
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 500, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(data)

Created a chunk of size 1077, which is longer than the specified 500
Created a chunk of size 1436, which is longer than the specified 500
Created a chunk of size 3405, which is longer than the specified 500


In [9]:
len(docs), docs

(30,
 [Document(page_content='Tabla de posiciones Eliminatorias Mundial 2026: Clasificación y resultados de la sexta fecha en Sudamérica | Marca \n\n\n \n\n \n\n\n \n \n\n\n  \n\n\n \n\nEs noticia: Espańa femeninoCopa del ReyVirginia FonsecaViniciusJosep MartínezLamineLamine YamalBernie EcclestoneCarvajalPartidos hoy Copa del Rey3IATLASBayern - Real Madrid dónde verSuecia - Espańa dónde verToledo - Sevilla dónde verMelissaAlcaraz - Norrie horaClasificación LigaNoticias hoyPrecio luz hoyFútbol hoy \n\nPortada de Marca\n\n\nEdición seleccionada\nES \n\n\nEdiciones:\n\nES (Espańa)\n\nEN (English)\n\n\nMX (México)\n\n\nUS (USA)\n\nEscoge tu edición de Marca.com favorita\n\n\nSiempre que entres en Marca.com se cargará esta edición\n\nES (Espańa)\n\n \n\nEN (English)\n\n \n\nMX (México)\n\n \n\nUS (USA)\n\n \n\n\nConfirmar\n\nSuscríbete\n\nFútbol\n\n\nBaloncesto\n\n\nMotor\n\n\nPolideportivo\n\n\nCoches\n\n\nÚltima hora\n\n\nFantasy\n\n\nAgenda TV\n\n\nMás\nCerrar\n\n21 de marzo\nde 2024Actu

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingrese la API Key de OpenAI : ")

## Cargando datos a Pinecone

In [11]:
#!pip install openai

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from pinecone import Pinecone as PineconeClient
from uuid import uuid4

# Inicializa Pinecone v3
api_key = getpass.getpass("Ingresa tu API Key de Pinecone : ")
index_name = "knowledge-base-eliminatorias"
pc = PineconeClient(api_key=api_key)
index = pc.Index(index_name)

# Embeddings
embedder = OpenAIEmbeddings()
texts = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

# Embed y subir
vectors = []
for i, text in enumerate(texts):
    embedding = embedder.embed_query(text)
    vectors.append({
        "id": str(uuid4()),
        "values": embedding,
        "metadata": metadatas[i] if i < len(metadatas) else {}
    })

index.upsert(vectors=vectors)
print("✅ Documentos insertados en Pinecone.")


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Documentos insertados en Pinecone.
